# Libraries

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Import Data

In [108]:
features = np.load('clean_data/features.npy')
labels = np.load('clean_data/labels.npy')

## Spilt Train, Cross Validation and Test Data

In [109]:
x_train, x_test_full, y_train, y_test_full = train_test_split(features, labels, train_size=0.6, random_state=1)
x_test, x_cv, y_test, y_cv = train_test_split(x_test_full, y_test_full, train_size=0.5, random_state=1)
print(x_train.shape)
print(x_cv.shape)
print(x_test.shape)
print(y_train.shape)

(33195, 33)
(11065, 33)
(11065, 33)
(33195,)


# Back Propogation, R  Back Propogation

# Activation Function

In [110]:
def sigmoid(Z): 
    return 1/(1+np.exp(-Z))

# Function to initialize weights as Gaussian distributions with specific mu and sigma

In [123]:
def initialize_weights(nodes_in, nodes_out, mu=0, sigma=0.1):
    return np.random.normal(mu, sigma, size=(nodes_in, nodes_out))

## Neural Network

In [208]:

class NeuralNetwork:
    def __init__(self, x, y,vocabulary_size=21361, nodes_in_layer1=60, nodes_in_layer2=30, nodes_in_layer3=1, l_rate=0.1):
        embedding_dim =60
        self.x=x
        vocab_size = vocabulary_size +1 # Add 1 for the out-of-vocabulary token
        self.embedding_weights = np.random.randn(vocab_size, embedding_dim)
        # define x, y
        self.inputs_in_layer0 =self.embedding(self.x)
        self.y = y.reshape(-1, 1)  # reshape y to be a column vector
        
        self.l_rate = l_rate  # learning rate
        
        # define and set the number of neurons in each layer
        self.nodes_in_layer1 = nodes_in_layer1
        self.nodes_in_layer2 = nodes_in_layer2
        self.nodes_in_layer3 = nodes_in_layer3
        
        # initialize the weights (theta) matrices
        self.thetas_layer0 = np.random.rand(self.inputs_in_layer0.shape[1] + 1, self.nodes_in_layer1) 
        self.thetas_layer1 = np.random.rand(self.nodes_in_layer1 + 1, self.nodes_in_layer2) 
        self.thetas_layer2 = np.random.rand(self.nodes_in_layer2 + 1, self.nodes_in_layer3)  
    def feedforward(self):      
        #compute all the nodes (a1, a2, a3, a4) in layer1
        n = self.inputs_in_layer0.shape[0]

        self.Z1 = self.thetas_layer0[0] + np.dot(self.inputs_in_layer0, self.thetas_layer0[1:])
        self.layer1 = sigmoid(self.Z1)  #values of a1, a2, a3, a4 in layer 1
        
        #compute all the nodes (a1, a2, a3) in layer2
        self.Z2 = self.thetas_layer1[0] + np.dot(self.layer1, self.thetas_layer1[1:])
        self.layer2 = sigmoid(self.Z2)  #values of a1, a2, a3 in layer 2
        
        #compute all the nodes (a1) in layer3
        self.Z3 = self.thetas_layer2[0] + np.dot(self.layer2, self.thetas_layer2[1:])
        self.layer3 = sigmoid(self.Z3)  #output layer      
        
        return self.layer3
    
    def cost_func(self):
        self.n = self.inputs_in_layer0.shape[0] #number of training examples
        self.cost = (1/self.n) * np.sum(-self.y * np.log(self.layer3) - (1 - self.y) * np.log(1 - self.layer3)) #cross entropy
        return self.cost 
    def embedding(self, x):
            self.embedded_input = self.embedding_weights[x]
            pooled_embeddings = np.mean(self.embedded_input, axis=1)
            return pooled_embeddings
    def calculate_accuracy(self ):
        actual_output=self.y
        predicted_output=self.layer3
        # Convert predicted probabilities to binary predictions (0 or 1)
        predicted_classes = (predicted_output >= 0.5).astype(int)

        # Compare predicted classes with actual classes
        correct_predictions = (predicted_classes == actual_output).sum()

        # Calculate accuracy
        accuracy = correct_predictions / len(actual_output)

        return accuracy

    def Rbackprop(self):
        # Define RProp parameters
        delta0 = 0.01  # Initial update value
        delta_max = 20  # Maximum update value
        delta_min = 1e-6  # Minimum update value
        eta_plus = 1.5  # Increase factor
        eta_minus = 0.5  # Decrease factor

        # dervative of E with respect to theta and bias in layer2
        self.dE_dlayer3 = (1 / self.n) * (self.layer3 - self.y) / (self.layer3 * (1 - self.layer3))
        self.dE_dZ3 = np.multiply(self.dE_dlayer3, sigmoid(self.Z3) * (1 - sigmoid(self.Z3)))
        self.dE_dtheta2 = np.dot(self.layer2.T, self.dE_dZ3)
        self.dE_dbias2 = np.dot(np.ones(self.n), self.dE_dZ3)

        # dervative of E with respect to theta and bias in layer1
        self.dE_dlayer2 = np.dot(self.dE_dZ3, self.thetas_layer2[1:].T)
        self.dE_dZ2 = np.multiply(self.dE_dlayer2, sigmoid(self.Z2) * (1 - sigmoid(self.Z2)))
        # self.dE_dZ1 = np.multiply(self.dE_dlayer2, dervative_relu(self.Z2))
        self.dE_dtheta1 = np.dot(self.layer1.T, self.dE_dZ2)
        self.dE_dbias1 = np.dot(np.ones(self.n), self.dE_dZ2)

        # dervative of E with respect to theta and bias in layer0
        self.dE_dlayer1 = np.dot(self.dE_dZ2, self.thetas_layer1[1:].T)
        self.dE_dZ1 = np.multiply(self.dE_dlayer1, sigmoid(self.Z1) * (1 - sigmoid(self.Z1)))
        # self.dE_dZ1 = np.multiply(self.dE_dlayer1, dervative_relu(self.Z1))
        self.dE_dtheta0 = np.dot(self.inputs_in_layer0.T, self.dE_dZ1)
        self.dE_dbias0 = np.dot(np.ones(self.n), self.dE_dZ1)

        # Initialize RProp update values
        if not hasattr(self, 'prev_dE_dtheta2'):
            self.prev_dE_dtheta2 = np.zeros_like(self.dE_dtheta2)
            self.delta_theta2 = np.full_like(self.dE_dtheta2, delta0)
        else:
            self.delta_theta2 = np.where(self.dE_dtheta2 * self.prev_dE_dtheta2 > 0,
                                        np.minimum(self.delta_theta2 * eta_plus, delta_max),
                                        np.maximum(self.delta_theta2 * eta_minus, delta_min))
        self.prev_dE_dtheta2 = self.dE_dtheta2

        if not hasattr(self, 'prev_dE_dtheta1'):
            self.prev_dE_dtheta1 = np.zeros_like(self.dE_dtheta1)
            self.delta_theta1 = np.full_like(self.dE_dtheta1, delta0)
        else:
            self.delta_theta1 = np.where(self.dE_dtheta1 * self.prev_dE_dtheta1 > 0,
                                        np.minimum(self.delta_theta1 * eta_plus, delta_max),
                                        np.maximum(self.delta_theta1 * eta_minus, delta_min))
        self.prev_dE_dtheta1 = self.dE_dtheta1

        if not hasattr(self, 'prev_dE_dtheta0'):
            self.prev_dE_dtheta0 = np.zeros_like(self.dE_dtheta0)
            self.delta_theta0 = np.full_like(self.dE_dtheta0, delta0)
        else:
            self.delta_theta0 = np.where(self.dE_dtheta0 * self.prev_dE_dtheta0 > 0,
                                        np.minimum(self.delta_theta0 * eta_plus, delta_max),
                                        np.maximum(self.delta_theta0 * eta_minus, delta_min))
        self.prev_dE_dtheta0 = self.dE_dtheta0

        # Updating theta using RProp in layers 2, 1, and 0
        self.thetas_layer2[1:] -= np.sign(self.dE_dtheta2) * self.delta_theta2
        self.thetas_layer1[1:] -= np.sign(self.dE_dtheta1) * self.delta_theta1
        self.thetas_layer0[1:] -= np.sign(self.dE_dtheta0) * self.delta_theta0

        # Updating bias using RProp in layers 2, 1, and 0
        self.thetas_layer2[0] -= np.sign(self.dE_dbias2) * self.delta_theta2[0]
        self.thetas_layer1[0] -= np.sign(self.dE_dbias1) * self.delta_theta1[0]
        self.thetas_layer0[0] -= np.sign(self.dE_dbias0) * self.delta_theta0[0]
        return self 
    
    def backprop(self):
        #dervative of E with respect to theta and bias in layer2
        self.dE_dlayer3 = (1/self.n) * (self.layer3-self.y)/(self.layer3*(1-self.layer3))
        self.dE_dZ3 = np.multiply(self.dE_dlayer3, (sigmoid(self.Z3)* (1-sigmoid(self.Z3))))
        self.dE_dtheta2 = np.dot(self.layer2.T, self.dE_dZ3)
        self.dE_dbias2 = np.dot(np.ones(self.n), self.dE_dZ3)
        
        #dervative of E with respect to theta and bias in layer1
        self.dE_dlayer2 = np.dot(self.dE_dZ3, self.thetas_layer2[1:].T)
        self.dE_dZ2 = np.multiply(self.dE_dlayer2, sigmoid(self.Z2)* (1-sigmoid(self.Z2)))
        self.dE_dtheta1 = np.dot(self.layer1.T, self.dE_dZ2)
        self.dE_dbias1 = np.dot(np.ones(self.n), self.dE_dZ2)
        
        #dervative of E with respect to theta and bias in layer0
        self.dE_dlayer1 = np.dot(self.dE_dZ2, self.thetas_layer1[1:].T)
        self.dE_dZ1 = np.multiply(self.dE_dlayer1, sigmoid(self.Z1)* (1-sigmoid(self.Z1)))
        self.dE_dtheta0 = np.dot(self.inputs_in_layer0.T, self.dE_dZ1)
        self.dE_dbias0 = np.dot(np.ones(self.n), self.dE_dZ1)
        #updating theta using gradient descent in layers 2, 1, and 0
        self.thetas_layer2[1:] = self.thetas_layer2[1:] - self.l_rate * self.dE_dtheta2
        self.thetas_layer1[1:] = self.thetas_layer1[1:] - self.l_rate * self.dE_dtheta1
        self.thetas_layer0[1:] = self.thetas_layer0[1:] - self.l_rate * self.dE_dtheta0
        # self.de_wegihts = np.dot(self.embedded_input.T, self.dE_dZ1)
        # self.embedding_weights = -self.embedding_weights - self.l_rate * self.de_wegihts

        
        #updating bias using gradient descent in layers 2, 1, and 0
        self.thetas_layer2[0] = self.thetas_layer2[0] - self.l_rate * self.dE_dbias2
        self.thetas_layer1[0] = self.thetas_layer1[0] - self.l_rate * self.dE_dbias1
        self.thetas_layer0[0] = self.thetas_layer0[0] - self.l_rate * self.dE_dbias0
        return self
    def fit(self,epochs,Backpropagate):
        losses=[]
        for i in range(epochs):
            self.feedforward()
            error=self.cost_func()
            losses.append(error)
            if Backpropagate==True:
                self.backprop()
            else:
                self.Rbackprop()
            print("iteration #",i+1)
            print('accuracy: ',self.calculate_accuracy())
            print("Cost: \n",error,"\n")
    def evaluate(self, x,y):
        inputs_layer0 = self.embedding(x)
        Z1 = self.thetas_layer0[0] + np.dot(inputs_layer0, self.thetas_layer0[1:])
        layer1 = sigmoid(Z1)

        Z2 = self.thetas_layer1[0] + np.dot(layer1, self.thetas_layer1[1:])
        layer2 = sigmoid(Z2)

        Z3 = self.thetas_layer2[0] + np.dot(layer2, self.thetas_layer2[1:])
        layer3 = sigmoid(Z3)
        loss= (1/inputs_layer0.shape[0]) * np.sum(-y * np.log(layer3) - (1 - y) * np.log(1 - layer3)) #cross entropy
        actual_output=y
        predicted_output=layer3
        # Convert predicted probabilities to binary predictions (0 or 1)
        predicted_classes = (predicted_output >= 0.5).astype(int)

        # Compare predicted classes with actual classes
        correct_predictions = (predicted_classes == actual_output).sum()

        # Calculate accuracy
        accuracy = correct_predictions / len(actual_output)
        return loss,accuracy,layer3          

# Train

In [209]:
nn=NeuralNetwork(x_train,y_train)
nn.fit(epochs=100,Backpropagate=True)

iteration # 1
accuracy:  0.4561229100768188
Cost: 
 6.8272322301306385 

iteration # 2
accuracy:  0.4561229100768188
Cost: 
 5.910436889736193 

iteration # 3
accuracy:  0.4561229100768188
Cost: 
 4.99373627675229 

iteration # 4
accuracy:  0.4561229100768188
Cost: 
 4.077545484909513 

iteration # 5
accuracy:  0.4561229100768188
Cost: 
 3.1640960364965376 

iteration # 6
accuracy:  0.4561229100768188
Cost: 
 2.265184971886224 

iteration # 7
accuracy:  0.4561229100768188
Cost: 
 1.4380177945862087 

iteration # 8
accuracy:  0.4561229100768188
Cost: 
 0.8706935103947374 

iteration # 9
accuracy:  0.4561229100768188
Cost: 
 0.7039672090653267 

iteration # 10
accuracy:  0.5438770899231812
Cost: 
 0.6900458301318018 

iteration # 11
accuracy:  0.5438770899231812
Cost: 
 0.6893311280685598 

iteration # 12
accuracy:  0.5438770899231812
Cost: 
 0.6892937814639241 

iteration # 13
accuracy:  0.5438770899231812
Cost: 
 0.6892917979360313 

iteration # 14
accuracy:  0.5438770899231812
Cost: 


In [210]:
test=nn.evaluate(x_cv,y_cv.reshape(-1,1))
print('Loss :',test[0])
print('Acc :',test[1])

Loss : 0.690561579228496
Acc : 0.5366470854044284


In [211]:
Test=nn.evaluate(x_test,y_test.reshape(-1,1))
print('Loss :',Test[0])
print('Acc :',Test[1])

Loss : 0.6899587685866853
Acc : 0.540081337550836


In [212]:
nn1=NeuralNetwork(x_train,y_train)
nn1.fit(epochs=100,Backpropagate=False)

iteration # 1
accuracy:  0.4561229100768188
Cost: 
 8.672378898961265 

iteration # 2
accuracy:  0.4561229100768188
Cost: 
 8.503777014071376 

iteration # 3
accuracy:  0.4561229100768188
Cost: 
 8.250873701830685 

iteration # 4
accuracy:  0.4561229100768188
Cost: 
 7.871482103587883 

iteration # 5
accuracy:  0.4561229100768188
Cost: 
 7.295060137484709 

iteration # 6
accuracy:  0.4561229100768188
Cost: 
 5.8905378363281775 

iteration # 7
accuracy:  0.4561229100768188
Cost: 
 3.2567309052839795 

iteration # 8
accuracy:  0.4561229100768188
Cost: 
 1.8779260392374009 

iteration # 9
accuracy:  0.45600241000150626
Cost: 
 0.7175000074122505 

iteration # 10
accuracy:  0.5438770899231812
Cost: 
 1.9241632901961079 

iteration # 11
accuracy:  0.5438770899231812
Cost: 
 0.9450100308529096 

iteration # 12
accuracy:  0.4561229100768188
Cost: 
 1.0690898964609068 

iteration # 13
accuracy:  0.5440277150173218
Cost: 
 0.69064283224657 

iteration # 14
accuracy:  0.4561229100768188
Cost: 
 

C:\Users\DELL\AppData\Local\Temp\ipykernel_23140\4003567598.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-Z))


iteration # 64
accuracy:  0.590751619219762
Cost: 
 0.6667485325902698 

iteration # 65
accuracy:  0.5904804940503088
Cost: 
 0.6659219478552831 

iteration # 66
accuracy:  0.5906009941256213
Cost: 
 0.6653281336771593 

iteration # 67
accuracy:  0.5900286187678867
Cost: 
 0.6657653269675723 

iteration # 68
accuracy:  0.5920771200482
Cost: 
 0.6643081107488705 

iteration # 69
accuracy:  0.5935231209519506
Cost: 
 0.6647686406501818 

iteration # 70
accuracy:  0.5932519957824973
Cost: 
 0.6633699240712567 

iteration # 71
accuracy:  0.5934929959331224
Cost: 
 0.6635039258871261 

iteration # 72
accuracy:  0.5931917457448411
Cost: 
 0.6623713270137145 

iteration # 73
accuracy:  0.5939147461967164
Cost: 
 0.6617209463864179 

iteration # 74
accuracy:  0.594728121705076
Cost: 
 0.6609973695752498 

iteration # 75
accuracy:  0.5962343726464829
Cost: 
 0.6599372896535085 

iteration # 76
accuracy:  0.5999397499623437
Cost: 
 0.659965695936053 

iteration # 77
accuracy:  0.5964151227594517

In [213]:
Test_1=nn1.evaluate(x_cv,y_cv.reshape(-1,1))
print('Loss :',Test_1[0])
print('Acc :',Test_1[1])

Loss : 0.6616096775268931
Acc : 0.5970176231360145


C:\Users\DELL\AppData\Local\Temp\ipykernel_23140\4003567598.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-Z))


In [214]:
Test_2=nn1.evaluate(x_test,y_test.reshape(-1,1))
print('Loss :',Test_2[0])
print('Acc :',Test_2[1])

Loss : 0.6625555547263093
Acc : 0.596023497514686


C:\Users\DELL\AppData\Local\Temp\ipykernel_23140\4003567598.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-Z))


# Bayiesn Neural Network

In [173]:
class BayesianNeuralNetwork:
    def __init__(self, x, y,vocabulary_size=21361,nodes_in_layer1=15, nodes_in_layer2=15, nodes_in_layer3=1, l_rate=0.001):
        embedding_dim= 60
        vocab_size = vocabulary_size +1 # Add 1 for the out-of-vocabulary token
        self.embedding_weights = np.random.randn(vocab_size, embedding_dim)
        # Define x, y
        self.inputs_in_layer0 = self.embedding(x) # Layer 0
        self.y = y.reshape(-1,1)

        self.l_rate = l_rate  # Learning rate

        # Define and set the number of neurons in each layer
        self.nodes_in_layer1 = nodes_in_layer1
        self.nodes_in_layer2 = nodes_in_layer2
        self.nodes_in_layer3 = nodes_in_layer3

        # Initialize weights and biases with smaller values using Gaussian distributions
        self.thetas_layer0 = initialize_weights(self.inputs_in_layer0.shape[1] + 1, self.nodes_in_layer1, mu=0.001, sigma=0.01)
        self.thetas_layer1 = initialize_weights(self.nodes_in_layer1 + 1, self.nodes_in_layer2, mu=0.001, sigma=0.01)
        self.thetas_layer2 = initialize_weights(self.nodes_in_layer2 + 1, self.nodes_in_layer3, mu=0.001, sigma=0.01)

        # Initialize prior distributions for weights
        self.prior_mean_theta0 = np.zeros_like(self.thetas_layer0)
        self.prior_mean_theta1 = np.zeros_like(self.thetas_layer1)
        self.prior_mean_theta2 = np.zeros_like(self.thetas_layer2)

        self.prior_variance_theta0 = np.ones_like(self.thetas_layer0)
        self.prior_variance_theta1 = np.ones_like(self.thetas_layer1)
        self.prior_variance_theta2 = np.ones_like(self.thetas_layer2)
    def embedding(self, x):
        self.embedded_input = self.embedding_weights[x]
        pooled_embeddings = np.mean(self.embedded_input, axis=1)
        return pooled_embeddings    

    def feedforward(self):
        # Sample weights from their respective Gaussian distributions for each forward pass
        self.Z1 = self.thetas_layer0[0] + np.dot(self.inputs_in_layer0, self.thetas_layer0[1:])
        self.layer1 = sigmoid(self.Z1)

        self.Z2 = self.thetas_layer1[0] + np.dot(self.layer1, self.thetas_layer1[1:])
        self.layer2 = sigmoid(self.Z2)

        self.Z3 = self.thetas_layer2[0] + np.dot(self.layer2, self.thetas_layer2[1:])
        self.layer3 = sigmoid(self.Z3)

        return self.layer3
    
    def calculate_accuracy(self):
        actual_output = self.y
        predicted_output = self.layer3
        # Convert predicted probabilities to binary predictions (0 or 1)
        predicted_classes = (predicted_output >= 0.5).astype(int)

        # Compare predicted classes with actual classes
        correct_predictions = (predicted_classes == actual_output).sum()

        # Calculate accuracy
        accuracy = correct_predictions / len(actual_output)

        return accuracy

    def log_prior(self):
        # Calculate log priors for weights using Gaussian distributions
        log_prior_theta0 = -0.5 * (np.log(2 * np.pi * self.prior_variance_theta0) +
                                   ((self.thetas_layer0 - self.prior_mean_theta0) ** 2) /
                                   self.prior_variance_theta0).sum()

        log_prior_theta1 = -0.5 * (np.log(2 * np.pi * self.prior_variance_theta1) +
                                   ((self.thetas_layer1 - self.prior_mean_theta1) ** 2) /
                                   self.prior_variance_theta1).sum()

        log_prior_theta2 = -0.5 * (np.log(2 * np.pi * self.prior_variance_theta2) +
                                   ((self.thetas_layer2 - self.prior_mean_theta2) ** 2) /
                                   self.prior_variance_theta2).sum()

        return log_prior_theta0 + log_prior_theta1 + log_prior_theta2

    def log_likelihood(self):
        # Compute log likelihood (negative log-likelihood for binary cross-entropy)
        self.n = self.inputs_in_layer0.shape[0]  # Number of training examples
        log_likelihood = (1 / self.n) * np.sum(-self.y * np.log(self.layer3) - (1 - self.y) * np.log(1 - self.layer3))
        return log_likelihood

    def log_posterior(self):
        # Compute log posterior using log prior and log likelihood
        log_prior = self.log_prior()
        log_likelihood = self.log_likelihood()
        log_posterior = log_prior + log_likelihood
        return log_posterior
    def perform_MCMC(self,proposal_variance=0.01):
         # Make a copy of the current weights for proposal
        proposed_thetas_layer0 = np.copy(self.thetas_layer0)
        proposed_thetas_layer1 = np.copy(self.thetas_layer1)
        proposed_thetas_layer2 = np.copy(self.thetas_layer2)

            # Perturb the weights for proposal (using a Gaussian random walk as an example)
        proposed_thetas_layer0 += np.random.normal(0, proposal_variance, size=self.thetas_layer0.shape)
        proposed_thetas_layer1 += np.random.normal(0, proposal_variance, size=self.thetas_layer1.shape)
        proposed_thetas_layer2 += np.random.normal(0, proposal_variance, size=self.thetas_layer2.shape)

            # Compute log-likelihoods for current and proposed weights
        current_log_likelihood = self.log_likelihood()

            # Compute log-posterior for the proposed weights
        self.thetas_layer0 = proposed_thetas_layer0
        self.thetas_layer1 = proposed_thetas_layer1
        self.thetas_layer2 = proposed_thetas_layer2

        proposed_log_likelihood = self.log_likelihood()
        proposed_log_posterior = self.log_prior() + proposed_log_likelihood

            # Accept or reject the proposal based on Metropolis-Hastings acceptance criterion
        acceptance_ratio = np.exp(proposed_log_posterior - current_log_likelihood)
        if np.random.uniform(0, 1) < acceptance_ratio:
            # Accept the proposal
            pass
        else:
            # Reject the proposal, revert weights to the previous state
            self.thetas_layer0 = np.copy(proposed_thetas_layer0)
            self.thetas_layer1 = np.copy(proposed_thetas_layer1)
            self.thetas_layer2 = np.copy(proposed_thetas_layer2)
    def fit(self,epochs):
        losses=[]
        for i in range(epochs):
            self.feedforward()
            error = self.log_likelihood()  # Compute log-likelihood as the error
            losses.append(error)
            self.perform_MCMC()
            print("iteration #",i+1)
            print('accuracy: ',self.calculate_accuracy())
            print("Cost: \n",error,"\n")
    def evaluate(self, x,y):
        inputs_layer0 = self.embedding(x)
        Z1 = self.thetas_layer0[0] + np.dot(inputs_layer0, self.thetas_layer0[1:])
        layer1 = sigmoid(Z1)

        Z2 = self.thetas_layer1[0] + np.dot(layer1, self.thetas_layer1[1:])
        layer2 = sigmoid(Z2)

        Z3 = self.thetas_layer2[0] + np.dot(layer2, self.thetas_layer2[1:])
        layer3 = sigmoid(Z3)
        loss= (1/inputs_layer0.shape[0]) * np.sum(-y * np.log(layer3) - (1 - y) * np.log(1 - layer3)) #cross entropy
        actual_output=y
        predicted_output=layer3
        # Convert predicted probabilities to binary predictions (0 or 1)
        predicted_classes = (predicted_output >= 0.5).astype(int)

        # Compare predicted classes with actual classes
        correct_predictions = (predicted_classes == actual_output).sum()

        # Calculate accuracy
        accuracy = correct_predictions / len(actual_output)
        return loss,accuracy,layer3          
    
   

# Train

In [174]:
bnn=BayesianNeuralNetwork(x_train,y_train)
bnn.fit(100)

iteration # 1
accuracy:  0.5438770899231812
Cost: 
 0.692676569338566 

iteration # 2
accuracy:  0.5438770899231812
Cost: 
 0.6920980133693211 

iteration # 3
accuracy:  0.4561229100768188
Cost: 
 0.6932188845250769 

iteration # 4
accuracy:  0.5438770899231812
Cost: 
 0.691942502964267 

iteration # 5
accuracy:  0.4561229100768188
Cost: 
 0.6944796803112536 

iteration # 6
accuracy:  0.4561229100768188
Cost: 
 0.6938006080882397 

iteration # 7
accuracy:  0.4561229100768188
Cost: 
 0.6932920064988984 

iteration # 8
accuracy:  0.5438770899231812
Cost: 
 0.6926895766925598 

iteration # 9
accuracy:  0.5438770899231812
Cost: 
 0.6911608051686295 

iteration # 10
accuracy:  0.5438770899231812
Cost: 
 0.6910810703905625 

iteration # 11
accuracy:  0.5438770899231812
Cost: 
 0.6905180658362565 

iteration # 12
accuracy:  0.5438770899231812
Cost: 
 0.6901909070230453 

iteration # 13
accuracy:  0.5438770899231812
Cost: 
 0.6904073994885646 

iteration # 14
accuracy:  0.5438770899231812
Cost

In [175]:
bnn_test=bnn.evaluate(x_cv,y_cv.reshape(-1,1))
print('Loss :',bnn_test[0])
print('Acc :',bnn_test[1])

Loss : 0.6926265915445428
Acc : 0.5366470854044284


In [176]:
bnn_test_1=bnn.evaluate(x_test,y_test.reshape(-1,1))
print('Loss :',bnn_test_1[0])
print('Acc :',bnn_test_1[1])

Loss : 0.6916558352708011
Acc : 0.540081337550836
